In [ ]:
pip install tensorflow-datasets

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'food101',
    split=['train', 'validation'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

In [ ]:
import numpy as np
import tensorflow as tf  # Corrected typo: isport -> import

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences  # Corrected import statement

In [ ]:
# Step 2: Load sample text data
data = "In the world of AI, machines are learning to understand natural language and generate human-like text.\n\
Deep learning models like LSTM are widely used in text generation tasks.\n\
The goal is to predict the next word in a sentence using previous words."

# Step 3: Text preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer # Make sure this import is here
tokenizer = Tokenizer()  # Corrected typo: Tokenizer() -> Tokenizer()
tokenizer.fit_on_texts([data])  # Corrected function name: Fit_on_texts -> fit_on_texts
total_words = len(tokenizer.word_index) + 1  # Corrected: added +1 to include the out-of-vocabulary token

In [ ]:
# Assume you have defined max_seq_len somewhere before this
# For example: max_seq_len = 10
max_seq_len = 10

# Step 4: Build LSTM model
model = Sequential()
model.add(Embedding(total_words, 10, input_length=max_seq_len-1))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))  # Corrected: activations -> activation, softmax' -> 'softmax'
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  # Corrected: compilofloss -> compile, nptimizer -> optimizer
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Example of how to prepare X and y:
sequences = []
for line in data.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        sequences.append(n_gram_sequence)

max_seq_len = max([len(x) for x in sequences])
input_sequences = np.array(pad_sequences(sequences, maxlen=max_seq_len, padding='pre'))

X = input_sequences[:,:-1]
y = input_sequences[:,-1]

y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

model.fit(X, y, epochs=300, verbose=1)  # Corrected: Fit -> fit


# Step 6: Text generation function
def generate_text(seed_text, next_words=10):
    for _ in range(next_words):  # Corrected: in -> _
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_seq_len - 1, padding='pre')
        predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)[0]

        output_word = ""  # Initialize output_word
        for word, index in tokenizer.word_index.items():
            if index == predicted:  # Corrected: = -> ==
                output_word = word
                break

        seed_text += " " + output_word  # Corrected: + -> " " +
    return seed_text

Epoch 1/300
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - accuracy: 0.0271 - loss: 3.5567
Epoch 2/300
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.0542 - loss: 3.5534
Epoch 3/300
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.0875 - loss: 3.5510
Epoch 4/300
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.1083 - loss: 3.5480
Epoch 5/300
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.1083 - loss: 3.5449
Epoch 6/300
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.0875 - loss: 3.5423 
Epoch 7/300
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.0875 - loss: 3.5394
Epoch 8/300
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.0875 - loss: 3.5337
Epoch 9/300
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.0979 - loss: 3.5268
Epoch 10/300
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.0875 - loss: 3.5225
Epoch 11/300
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.1083 - loss: 3.5106
Epoch 12/300
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.1083 - l

In [ ]:
# Step 7: Generate new text
seed_text = "In the world"  # Provide a starting seed text
generated_text = generate_text(seed_text, next_words=10)  # Generate 10 words after the seed text
print(generated_text)  # Print the generated text

In the world of ai machines are learning to understand natural language and
